In [1]:
import os
import re
import json
import requests

from bs4 import BeautifulSoup, Tag
from urllib.parse import urljoin, unquote

BASE_URL = 'https://uk.wikipedia.org'
MAIN_URL = BASE_URL + '/wiki/%D0%92%D1%96%D0%BA%D1%96%D0%BF%D0%B5%D0%B4%D1%96%D1%8F:%D0%9F%D1%80%D0%BE%D1%94%D0%BA%D1%82:%D0%A7%D0%B8_%D0%B2%D0%B8_%D0%B7%D0%BD%D0%B0%D1%94%D1%82%D0%B5/%D0%90%D1%80%D1%85%D1%96%D0%B2_%D1%80%D1%83%D0%B1%D1%80%D0%B8%D0%BA%D0%B8'
OUTPUT_DIR = 'data/uk'


def get_year_quarter_links_from_archive(main_page_url):
    resp = requests.get(main_page_url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')

    div = soup.find('div', class_='mw-heading mw-heading2 ext-discussiontools-init-section')
    archive_ul = div.find_next('ul')

    archive = {}
    for li in archive_ul.find_all('li', recursive=False):
        year_b = li.find('b')
        year = year_b.text.strip().split()[0]

        quarters = []
        for a in li.find_all('a', recursive=False):
            quarter_name = a.text.strip()
            href = a.get('href')
            full_url = BASE_URL + href
            exists = 'new' not in a.get('class', [])

            quarters.append({
                'quarter': quarter_name,
                'url':   full_url,
                'exists': exists
            })
        
        archive[year] = quarters

    return archive


In [2]:
archive = get_year_quarter_links_from_archive(MAIN_URL)

archive

{'2006': [{'quarter': 'січень—березень',
   'url': 'https://uk.wikipedia.org/wiki/%D0%92%D1%96%D0%BA%D1%96%D0%BF%D0%B5%D0%B4%D1%96%D1%8F:%D0%9F%D1%80%D0%BE%D1%94%D0%BA%D1%82:%D0%A7%D0%B8_%D0%B2%D0%B8_%D0%B7%D0%BD%D0%B0%D1%94%D1%82%D0%B5/%D0%90%D1%80%D1%85%D1%96%D0%B2_%D1%80%D1%83%D0%B1%D1%80%D0%B8%D0%BA%D0%B8/2006-01',
   'exists': True},
  {'quarter': 'квітень—червень',
   'url': 'https://uk.wikipedia.org/wiki/%D0%92%D1%96%D0%BA%D1%96%D0%BF%D0%B5%D0%B4%D1%96%D1%8F:%D0%9F%D1%80%D0%BE%D1%94%D0%BA%D1%82:%D0%A7%D0%B8_%D0%B2%D0%B8_%D0%B7%D0%BD%D0%B0%D1%94%D1%82%D0%B5/%D0%90%D1%80%D1%85%D1%96%D0%B2_%D1%80%D1%83%D0%B1%D1%80%D0%B8%D0%BA%D0%B8/2006-02',
   'exists': True},
  {'quarter': 'липень—вересень',
   'url': 'https://uk.wikipedia.org/wiki/%D0%92%D1%96%D0%BA%D1%96%D0%BF%D0%B5%D0%B4%D1%96%D1%8F:%D0%9F%D1%80%D0%BE%D1%94%D0%BA%D1%82:%D0%A7%D0%B8_%D0%B2%D0%B8_%D0%B7%D0%BD%D0%B0%D1%94%D1%82%D0%B5/%D0%90%D1%80%D1%85%D1%96%D0%B2_%D1%80%D1%83%D0%B1%D1%80%D0%B8%D0%BA%D0%B8/2006-03',
   'exists': 

In [41]:
def parse_quarter_facts(quarter_url: str, default_section_name: str) -> list[dict]:
    resp = requests.get(quarter_url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    results: list[dict] = []

    # Find all section divs that act as headers for facts
    section_divs = soup.find_all("div", class_="mw-heading mw-heading2 ext-discussiontools-init-section")
    
    is_simple_case = len(section_divs) == 1

    for div in section_divs:
        header = div.find('h2')
        section_name_text = header.get_text(strip=True) if header else ""

        # The rule is to skip the div with this specific title, and all its contents.
        if section_name_text == "Архів по місяцях" and not is_simple_case:
            continue

        section_name = default_section_name
        # We extract titles unless it's the simple, single-div case.
        if not is_simple_case and section_name_text and section_name_text != "Архів по місяцях":
            section_name = section_name_text

        uls_for_this_section = []
        # Collect all 'ul' siblings that appear after the current 'div' and before the next one
        for sibling in div.find_next_siblings():
            if sibling.name == 'div' and "mw-heading" in sibling.get('class', []):
                break
            if sibling.name == 'ul':
                uls_for_this_section.append(sibling)

        # In the original simple case (only one section div on the page), skip the first 'ul'
        if is_simple_case and uls_for_this_section:
            uls_to_process = uls_for_this_section[1:]
        else:
            uls_to_process = uls_for_this_section

        for ul in uls_to_process:
            for li in ul.find_all('li', recursive=False):
                fact_text = li.get_text(" ", strip=True).replace('\xa0', ' ').strip()

                links: list[str] = []
                relevant_links: list[str] = []

                for a in li.find_all("a", href=True):
                    href = a["href"]
                    if not href.startswith("/wiki/"):
                        continue

                    full_url = unquote(urljoin(quarter_url, href))

                    if a.find_parent('b') or a.find('b'):
                        relevant_links.append(full_url)

                    links.append(full_url)

                results.append({
                    "section": section_name,
                    "text": fact_text,
                    "links": links,
                    "relevant_links": relevant_links,
                })

    return results

In [46]:
year = '2025'
quarter = 1
quarter_url = archive[year][quarter]['url']
default_section_name = archive[year][quarter]['quarter'] + ' ' + year

facts = parse_quarter_facts(quarter_url, default_section_name)
facts

[{'section': '13 квітня — 12 травня 2025',
  'text': 'Автор першого українського посібника для гри на гітарі відомий також як композитор, що написав музику до гімну України',
  'links': ['https://uk.wikipedia.org/wiki/Вербицький_Михайло_Михайлович',
   'https://uk.wikipedia.org/wiki/Гітара',
   'https://uk.wikipedia.org/wiki/Гімн_України'],
  'relevant_links': ['https://uk.wikipedia.org/wiki/Вербицький_Михайло_Михайлович']},
 {'section': '13 квітня — 12 травня 2025',
  'text': 'На картині «Нічна варта» (на мал.) зображена денна сцена — просто лак, що вкриває зображення, потемнів з часом',
  'links': ['https://uk.wikipedia.org/wiki/Нічна_варта_(картина)'],
  'relevant_links': ['https://uk.wikipedia.org/wiki/Нічна_варта_(картина)']},
 {'section': '13 квітня — 12 травня 2025',
  'text': '«Бабуся спортивної гімнастики» вперше стала чемпіонкою світу ще за часів СРСР , але й у майже 50 років перемагає втричі молодших суперниць',
  'links': ['https://uk.wikipedia.org/wiki/Чусовітіна_Оксана_Ол

In [53]:
MONTHS = {
    'січень': 'січень', 'січня': 'січень',
    'лютий': 'лютий', 'лютого': 'лютий',
    'березень': 'березень', 'березня': 'березень',
    'квітень': 'квітень', 'квітня': 'квітень',
    'травень': 'травень', 'травня': 'травень',
    'червень': 'червень', 'червня': 'червень',
    'липень': 'липень', 'липня': 'липень',
    'серпень': 'серпень', 'серпня': 'серпень',
    'вересень': 'вересень', 'вересня': 'вересень',
    'жовтень': 'жовтень', 'жовтня': 'жовтень',
    'листопад': 'листопад', 'листопада': 'листопад',
    'грудень': 'грудень', 'грудня': 'грудень',
}

def extract_month_from_title(title: str) -> str | None:
    """Extracts the first mentioned month from a section title."""
    words = re.split(r'[\s—-]+', title)

    for word in words:
        cleaned_word = word.lower().strip()
        
        if cleaned_word in MONTHS:
            return MONTHS[cleaned_word]

    return None

In [60]:
extract_month_from_title('3 22 грудня 2022 — 23 січня 2023')

'грудень'

In [52]:
import dateparser

year = '2025'
month = '7'
section = '11 — 25 лютого 2025'

dt_month = dateparser.parse(section)
fact_date = dt_month.strftime("%Y-%m") if dt_month else None

fact_date

'2025-02'